In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
!pip install fastai

In [3]:
#For example, so not exported
from fastai.vision.core import *
from fastai.vision.data import *
from fastai.vision.all import *

In [4]:
from pathlib import Path

In [5]:
import numpy as np

In [6]:
import os
import shutil
import random
os.environ['FASTAI_TB_BACKEND'] = 'pytorch'

In [7]:
!export FASTAI_TB_BACKEND='auto'

### Data

In [8]:
path = Path('dataset')

# Get all the image files in the dataset directory
items = get_image_files(path)

# Define the splitter
splitter = GrandparentSplitter(train_name='Train', valid_name='Valid')
splits = splitter(items)

# Use the splits to create the training and validation sets
train_items = items[splits[0]]
valid_items = items[splits[1]]

In [ ]:
print(train_items)
print(valid_items)

In [9]:
np.random.seed(41)

# Define the DataBlock
dblock = DataBlock(blocks=(ImageBlock, CategoryBlock),
                   get_items=get_image_files,
                   splitter=GrandparentSplitter(train_name='Train', valid_name='Valid'),
                   get_y=parent_label,
                   item_tfms=Resize(256),
                   batch_tfms=aug_transforms(size=256, min_scale=0.75))

# Create the DataLoaders
dls = dblock.dataloaders(path)

# Show a batch of data
dls.show_batch()

Could not do one pass in your dataloader, there is something wrong in it. Please see the stack trace below:


RuntimeError: The MPS backend is supported on MacOS 12.3+.Current OS version can be queried using `sw_vers`

In [ ]:
data.classes, data.c, len(data.train_ds), len(data.valid_ds)

In [ ]:
print("Number of examples in training:", len(data.train_ds))
print("Number of examples in validation:", len(data.valid_ds))

In [ ]:
xb,yb = data.one_batch()
xb.shape,yb.shape

---

In [ ]:
data.show_batch(rows=3, figsize=(10,10))

## DarkCovidNet (modified Darknet model)

In [ ]:
def conv_block(ni, nf, size=3, stride=1):
    for_pad = lambda s: s if s > 2 else 3
    return nn.Sequential(
        nn.Conv2d(ni, nf, kernel_size=size, stride=stride,
                  padding=(for_pad(size) - 1)//2, bias=False), 
        nn.BatchNorm2d(nf),
        nn.LeakyReLU(negative_slope=0.1, inplace=True)  
    )

In [ ]:
def triple_conv(ni, nf):
    return nn.Sequential(
        conv_block(ni, nf),
        conv_block(nf, ni, size=1),  
        conv_block(ni, nf)
    )

In [ ]:
def maxpooling():
    return nn.MaxPool2d(2, stride=2)

In [ ]:
model = nn.Sequential(
    conv_block(3, 8),
    maxpooling(),
    conv_block(8, 16),
    maxpooling(),
    triple_conv(16, 32),
    maxpooling(),
    triple_conv(32, 64),
    maxpooling(),
    triple_conv(64, 128),
    maxpooling(),
    triple_conv(128, 256),
    conv_block(256, 128, size=1),
    conv_block(128, 256),
    conv_layer(256, 3),
    Flatten(),
    nn.Linear(507, 3)
)

In [ ]:
learn = Learner(data, model, loss_func = nn.CrossEntropyLoss(), metrics=accuracy)

In [ ]:
print(learn.summary())

In [ ]:
learn.fit_one_cycle(100, max_lr=3e-3)

In [ ]:
learn.recorder.plot_losses()

In [ ]:
print("Number of examples in testing:", len(data.valid_ds))

In [ ]:
probs,targets = learn.get_preds(ds_type=DatasetType.Valid) # Predicting without TTA

In [ ]:
accuracy(probs,targets)

In [ ]:
data.classes, data.c, len(data.train_ds), len(data.valid_ds)

In [ ]:
probs = np.argmax(probs, axis=1)
correct = 0
for idx, pred in enumerate(probs):
    if pred == targets[idx]:
        correct += 1
accuracy = correct / len(probs)
print(len(probs), correct, accuracy)

from sklearn.metrics import confusion_matrix
np.set_printoptions(threshold=np.inf) # shows whole confusion matrix
cm1 = confusion_matrix(targets, probs)
print(cm1)

from sklearn.metrics import classification_report
y_true1 = targets
y_pred1 = probs
target_names = ['Covid-19', 'No_findings', 'Pneumonia']
print(classification_report(y_true1, y_pred1, target_names=target_names))

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)

In [ ]:
cm_fig = interp.plot_confusion_matrix(return_fig=True)
ax = cm_fig.gca()
ax.set_ylim(interp.data.c - .5, - .5);